# Alt-Baltimore Population Estimation (Notebook 04)

In this notebook we estimate:

- The population living inside the **Alt-Baltimore districts**
- The population of the **same geographic footprint**, using real-world Census tract data
- Per-district population and density (people per square mile)

Data sources:

- Alt-Baltimore polygons: GeoJSON exported from Google Earth
- Census tracts: 2024 TIGER/Line shapefile for Maryland (tl_2024_24_tract)
- Population: ACS 5-year table B01003 (Total Population) for Maryland tracts


Imports & Paths

In [46]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Set base paths relative to the notebook
BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
REF_DIR  = DATA_DIR / "reference"
POLY_DIR = DATA_DIR / "polygons"

ALT_POLYGONS_PATH = POLY_DIR / "alt_baltimore_polygons.geojson"
TRACTS_SHP_PATH   = REF_DIR / "tl_2024_24_tract" / "tl_2024_24_tract.shp"
ACS_POP_PATH = REF_DIR / "acs2023_5yr_B01003_04000US24.csv"

# ACS population column (Census Reporter B01003)
POP_COL = "B01003_001E"


Load Alt-Baltimore Polygons

In [47]:
alt_polygons = gpd.read_file(ALT_POLYGONS_PATH)

print("Alt-Baltimore districts:", len(alt_polygons))
print("Columns:", alt_polygons.columns.tolist())
print("CRS:", alt_polygons.crs)

alt_polygons.head()


Alt-Baltimore districts: 428
Columns: ['name', 'geometry']
CRS: EPSG:4326


,name,geometry
0,Pratt Tech,"POLYGON ((-76.60898 39.24015, -76.61016 39.239..."
1,Aurum Hill,"POLYGON ((-76.62296 39.22965, -76.62405 39.225..."
2,Jones Family Community Tower,"POLYGON ((-76.61841 39.31124, -76.61841 39.311..."
3,UB Golf Course,"POLYGON ((-76.64735 39.2716, -76.64168 39.2756..."
4,Baltimore Central Station,"POLYGON ((-76.61627 39.30856, -76.6162 39.3072..."


Load Tracts + ACS Population

In [48]:
# Load Maryland tracts from TIGER/Line shapefile
tracts = gpd.read_file(TRACTS_SHP_PATH)
print("Total MD tracts:", len(tracts))
print("Tracts CRS:", tracts.crs)
tracts.head()


Total MD tracts: 1475
Tracts CRS: EPSG:4269


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,24,005,403100,24005403100,1400000US24005403100,4031,Census Tract 4031,G5020,S,1913598,0,+39.3530950,-076.7333148,"POLYGON ((-76.74491 39.35986, -76.74431 39.359..."
1,24,005,403201,24005403201,1400000US24005403201,4032.01,Census Tract 4032.01,G5020,S,1764534,0,+39.3549038,-076.7215235,"POLYGON ((-76.73369 39.36204, -76.7329 39.3620..."
2,24,033,807304,24033807304,1400000US24033807304,8073.04,Census Tract 8073.04,G5020,S,1776714,0,+39.0249780,-076.9594342,"POLYGON ((-76.97171 39.01858, -76.97164 39.018..."
3,24,033,807305,24033807305,1400000US24033807305,8073.05,Census Tract 8073.05,G5020,S,3030479,4428,+39.0121779,-076.9635510,"POLYGON ((-76.97728 39.01095, -76.97714 39.011..."
4,24,033,807405,24033807405,1400000US24033807405,8074.05,Census Tract 8074.05,G5020,S,7093435,27200,+39.0383526,-076.9283090,"POLYGON ((-76.94986 39.02208, -76.94944 39.024..."


In [49]:
# Load ACS population CSV from Census Reporter (B01003 for MD tracts)
acs = pd.read_csv(ACS_POP_PATH)
acs.head()


,geoid,name,B01003001,"B01003001, Error"
0,04000US24,Maryland,6170738,NaN


In [69]:
# Reload the Maryland Census Data from your reference folder
md = pd.read_csv(REF_DIR / "Maryland_Census_Data.csv")

# Make sure CNTY2010 and CT10 are strings with the right zero-padding
md["CNTY2010"] = md["CNTY2010"].astype(str).str.zfill(5)   # e.g. 24005, 24510
md["CT10"]     = md["CT10"].astype(str).str.zfill(6)       # e.g. 040310, 073704

# Build an 11-digit GEOID that matches the TIGER shapefile
md["GEOID"] = md["CNTY2010"] + md["CT10"]

print("Sample MD GEOIDs:", md["GEOID"].head().tolist())


Sample MD GEOIDs: ['24029950100', '24029950200', '24029950300', '24029950500', '24029950400']


Clean ACS GEOIDs and Attach Population to Tracts

In [70]:
print("ACS columns:", acs.columns.tolist())

# Rename population field for easier reference
acs = acs.rename(columns={"B01003001": "total_pop"})

# Keep only the columns we care about
acs_tracts = acs[["geoid", "total_pop"]].copy()

acs_tracts.head()



ACS columns: ['geoid', 'name', 'total_pop', 'B01003001, Error']


,geoid,total_pop
0,04000US24,6170738


In [71]:
# --- Clean ACS GEOIDs and Attach Population to Tracts (Census API style) ---

print("ACS columns:", acs.columns.tolist())
# should show: ['geoid', 'name', 'B01003001', 'B01003001, Error']

# Keep just the GEOID + population, and align column names to the tracts GeoDataFrame
acs_tracts = (
    acs
    .rename(columns={
        "geoid": "GEOID",     # match tl_2024_24_tract.shp
        POP_COL: "total_pop"  # POP_COL = "B01003_001E"
    })
    [["GEOID", "total_pop"]]
)

acs_tracts.head()



ACS columns: ['geoid', 'name', 'total_pop', 'B01003001, Error']


,GEOID,total_pop
0,04000US24,6170738


In [73]:
# --- Merge population onto tracts ---

tracts = tracts.merge(acs_tracts, on="GEOID", how="left")

print("Tracts with population:", tracts["total_pop"].notna().sum())
tracts.head()


Tracts with population: 0


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,total_pop_x,SQ_MILES_x,POP100_x,HU100_x,NHW_x,NHB_x,NHAI_x,NHA_x,NHNH_x,NHO_x,NHT_x,HISP_x,PNHW_x,PNHB_x,PNHAI_x,PNHA_x,PNHNH_x,PNHO_x,PNHT_x,PHISP_x,MEDAGE_x,OBJECTID_1,FID_TL_201,ALAND10,AWATER10,CNTY2010,CT10,CT2010,GEODESC,ACRES,SQ_MILES_y,POP100_y,HU100_y,NHW_y,NHB_y,NHAI_y,NHA_y,NHNH_y,NHO_y,NHT_y,HISP_y,PNHW_y,PNHB_y,PNHAI_y,PNHA_y,PNHNH_y,PNHO_y,PNHT_y,PHISP_y,POP65_,PCTPOP65_,MEDAGE_y,VACNS,PVACNS,PHOWN,PWOMORT,PRENT,PLT18SP,CNTY20101,CT101,AREA,ACRES_1,SQMILES,PSQM,REPORT_2_P,REPORT_9_P,Shape_Length,Shape_Area,population,total_pop_y,total_pop
0,24,005,403100,24005403100,1400000US24005403100,4031,Census Tract 4031,G5020,S,1913598,0,+39.3530950,-076.7333148,"POLYGON ((-76.74491 39.35986, -76.74431 39.359...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,492.0,496.0,1913333.0,0.0,24005.0,403100.0,2.400540e+10,4031.00,472.77936,0.73872,2462.0,952.0,720.0,1591.0,4.0,22.0,0.0,7.0,44.0,74.0,29.2,64.6,0.2,0.9,0.0,0.3,1.8,3.0,730.0,29.7,51.6,36.0,3.8,83.4,23.2,16.6,40.0,24005.0,403100.0,0.0,472.77936,0.73872,3332.81,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,9614.294520,3.204235e+06,0,NaN,NaN
1,24,005,403201,24005403201,1400000US24005403201,4032.01,Census Tract 4032.01,G5020,S,1764534,0,+39.3549038,-076.7215235,"POLYGON ((-76.73369 39.36204, -76.7329 39.3620...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,549.0,553.0,1764535.0,0.0,24005.0,403201.0,2.400540e+10,4032.01,436.01019,0.68127,3076.0,1191.0,281.0,2563.0,24.0,33.0,2.0,3.0,65.0,105.0,9.1,83.3,0.8,1.1,0.1,0.1,2.1,3.4,481.0,15.6,44.2,47.0,3.9,92.5,18.3,7.5,48.3,24005.0,403201.0,0.0,436.01019,0.68127,4515.13,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,8352.671252,2.955185e+06,0,NaN,NaN
2,24,033,807304,24033807304,1400000US24033807304,8073.04,Census Tract 8073.04,G5020,S,1776714,0,+39.0249780,-076.9594342,"POLYGON ((-76.97171 39.01858, -76.97164 39.018...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,769.0,773.0,1779313.0,0.0,24033.0,807304.0,2.403381e+10,8073.04,439.63469,0.68693,1928.0,625.0,462.0,522.0,9.0,169.0,0.0,2.0,62.0,702.0,24.0,27.1,0.5,8.8,0.0,0.1,3.2,36.4,291.0,15.1,39.2,37.0,5.9,91.3,23.1,8.7,24.7,24033.0,807304.0,0.0,439.63469,0.68693,2806.70,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,7884.644364,2.952312e+06,0,NaN,NaN
3,24,033,807305,24033807305,1400000US24033807305,8073.05,Census Tract 8073.05,G5020,S,3030479,4428,+39.0121779,-076.9635510,"POLYGON ((-76.97728 39.01095, -76.97714 39.011...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453.0,457.0,3078040.0,4428.0,24033.0,807305.0,2.403381e+10,8073.05,761.61967,1.19003,3676.0,1116.0,478.0,1230.0,8.0,400.0,3.0,10.0,70.0,1477.0,13.0,33.5,0.2,10.9,0.1,0.3,1.9,40.2,356.0,9.7,33.1,52.0,4.7,64.9,16.0,35.1,31.1,24033.0,807305.0,0.0,761.61967,1.19003,3089.00,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,9375.156855,5.112749e+06,0,NaN,NaN
4,24,033,807405,24033807405,1400000US24033807405,8074.05,Census Tract 8074.05,G5020,S,7093435,27200,+39.0383526,-076.9283090,"POLYGON ((-76.94986 39.02208, -76.94944 39.024...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,768.0,772.0,7092548.0,27200.0,24033.0,807405.0,2.403381e+10,8074.05,1759.15854,2.74869,5858.0,2319.0,2458.0,1244.0,11.0,611.0,0.0,34.0,139.0,1361.0,42.0,21.2,0.2,10.4,0.0,0.6,2.4,23.2,673.0,11.5,35.5,154.0,6.6,64.8,24.2,35.2,23.2,24033.0,807405.0,0.0,1759.15854,2.74869,2131.20,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,14194.627641,1.181656e+07,0,NaN,NaN


In [74]:
md = pd.read_csv(REF_DIR / "Maryland_Census_Data.csv")
pd.set_option('display.max_columns', None)
md.head()



,OBJECTID_1,FID_TL_201,ALAND10,AWATER10,CNTY2010,CT10,CT2010,GEODESC,ACRES,SQ_MILES,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,POP65_,PCTPOP65_,MEDAGE,VACNS,PVACNS,PHOWN,PWOMORT,PRENT,PLT18SP,CNTY20101,CT101,AREA,ACRES_1,SQMILES,PSQM,REPORT_2_P,REPORT_9_P,Shape_Length,Shape_Area
0,1,0,170616683,5541577,24029,950100,24029950100,9501.0,42392.60956,66.23845,3838,1796,3143,299,5,37,0,2,47,305,81.9,7.8,0.1,1.0,0.0,0.1,1.2,7.9,620,16.2,42.8,154,8.6,77.7,30.9,22.3,27.8,24029,950100,0,42392.60956,66.23845,57.94,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,138814.599164,2.869601e+08
1,2,1,279186393,29289092,24029,950200,24029950200,9502.0,69542.40176,108.66000,4844,2640,3767,737,4,14,0,4,58,260,77.8,15.2,0.1,0.3,0.0,0.1,1.2,5.4,946,19.5,46.2,223,8.4,79.0,34.0,21.0,30.8,24029,950200,0,69542.40176,108.66000,44.58,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,268980.601510,4.706709e+08
2,3,2,21445745,1762900,24029,950300,24029950300,9503.0,5332.72370,8.33238,5775,2574,4226,1085,5,95,1,9,107,247,73.2,18.8,0.1,1.6,0.0,0.2,1.9,4.3,1341,23.2,36.9,307,11.9,51.7,34.6,48.3,47.0,24029,950300,0,5332.72370,8.33238,693.08,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,47165.031258,3.601589e+07
3,4,3,72736967,56708716,24029,950500,24029950500,9505.0,18197.97926,28.43434,2623,1770,2307,236,2,4,0,0,35,39,88.0,9.0,0.1,0.2,0.0,0.0,1.3,1.5,715,27.3,52.3,151,8.5,77.1,40.5,22.9,37.0,24029,950500,0,18197.97926,28.43434,92.25,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,225444.550101,1.225410e+08
4,5,4,173519483,33126235,24029,950400,24029950400,9504.0,43502.34671,67.97242,3117,1769,2340,646,7,7,2,3,56,56,75.1,20.7,0.2,0.2,0.1,0.1,1.8,1.8,775,24.9,50.9,154,8.7,78.3,39.7,21.7,38.1,24029,950400,0,43502.34671,67.97242,45.86,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,246617.820478,2.935438e+08


In [75]:
race = pd.read_csv(REF_DIR / "Census_2020_Pop_by_Race.csv")
md   = pd.read_csv(REF_DIR / "Maryland_Census_Data.csv")

print("race columns:", race.columns.tolist())
print("md columns:", md.columns.tolist())


race columns: ['Label (Grouping)', 'Census Tract 7301.02, Anne Arundel County, Maryland', 'Census Tract 7501.01, Anne Arundel County, Maryland', 'Census Tract 7501.02, Anne Arundel County, Maryland', 'Census Tract 7502.01, Anne Arundel County, Maryland', 'Census Tract 7502.04, Anne Arundel County, Maryland', 'Census Tract 7503, Anne Arundel County, Maryland', 'Census Tract 7504, Anne Arundel County, Maryland', 'Census Tract 7508.01, Anne Arundel County, Maryland', 'Census Tract 7508.03, Anne Arundel County, Maryland', 'Census Tract 7508.04, Anne Arundel County, Maryland', 'Census Tract 7511.02, Anne Arundel County, Maryland', 'Census Tract 7512, Anne Arundel County, Maryland', 'Census Tract 9800, Anne Arundel County, Maryland', 'Census Tract 4001, Baltimore County, Maryland', 'Census Tract 4002, Baltimore County, Maryland', 'Census Tract 4004, Baltimore County, Maryland', 'Census Tract 4006, Baltimore County, Maryland', 'Census Tract 4007.01, Baltimore County, Maryland', 'Census Tract 

In [76]:
print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)
print("REF_DIR:", REF_DIR)
list(REF_DIR.glob("*"))


BASE_DIR: /Users/warrenjones/Dev/alt-baltimore-geospatial
DATA_DIR: /Users/warrenjones/Dev/alt-baltimore-geospatial/data
REF_DIR: /Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference


[PosixPath('/Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference/.DS_Store'),
 PosixPath('/Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference/tl_2024_24_tract'),
 PosixPath('/Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference/Maryland_Census_Data.csv'),
 PosixPath('/Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference/Neighborhood_Statistical_Area_(NSA)_Boundaries.geojson'),
 PosixPath('/Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference/acs2023_5yr_B01003_04000US24.csv'),
 PosixPath('/Users/warrenjones/Dev/alt-baltimore-geospatial/data/reference/Census_2020_Pop_by_Race.csv')]

In [33]:
print("race columns:", race.columns.tolist())
print("md columns:", md.columns.tolist())


race columns: ['Label (Grouping)', 'Census Tract 7301.02, Anne Arundel County, Maryland', 'Census Tract 7501.01, Anne Arundel County, Maryland', 'Census Tract 7501.02, Anne Arundel County, Maryland', 'Census Tract 7502.01, Anne Arundel County, Maryland', 'Census Tract 7502.04, Anne Arundel County, Maryland', 'Census Tract 7503, Anne Arundel County, Maryland', 'Census Tract 7504, Anne Arundel County, Maryland', 'Census Tract 7508.01, Anne Arundel County, Maryland', 'Census Tract 7508.03, Anne Arundel County, Maryland', 'Census Tract 7508.04, Anne Arundel County, Maryland', 'Census Tract 7511.02, Anne Arundel County, Maryland', 'Census Tract 7512, Anne Arundel County, Maryland', 'Census Tract 9800, Anne Arundel County, Maryland', 'Census Tract 4001, Baltimore County, Maryland', 'Census Tract 4002, Baltimore County, Maryland', 'Census Tract 4004, Baltimore County, Maryland', 'Census Tract 4006, Baltimore County, Maryland', 'Census Tract 4007.01, Baltimore County, Maryland', 'Census Tract 

In [77]:
# --- Build GEOID in md to match TIGER tracts ---

STATE_FIPS = "24"  # Maryland

# county: 3-digit, zero-padded
md["COUNTYFP"] = md["CNTY2010"].astype(str).str.zfill(3)

# tract: strip decimal, zero-pad to 6 digits (e.g. 803.01 -> '080301')
tract_raw = md["CT2010"].astype(str).str.strip()
tract_clean = tract_raw.str.replace(".", "", regex=False)
md["TRACTCE"] = tract_clean.str.zfill(6)

# final 11-digit GEOID
md["GEOID"] = STATE_FIPS + md["COUNTYFP"] + md["TRACTCE"]

md[["CNTY2010", "CT2010", "COUNTYFP", "TRACTCE", "GEOID"]].head()


,CNTY2010,CT2010,COUNTYFP,TRACTCE,GEOID
0,24029,24029950100,24029,24029950100,242402924029950100
1,24029,24029950200,24029,24029950200,242402924029950200
2,24029,24029950300,24029,24029950300,242402924029950300
3,24029,24029950500,24029,24029950500,242402924029950500
4,24029,24029950400,24029,24029950400,242402924029950400


In [78]:
md_cols_keep = [
    "GEOID",
    "SQ_MILES",
    "POP100", "HU100",
    "NHW", "NHB", "NHAI", "NHA", "NHNH", "NHO", "NHT", "HISP",
    "PNHW", "PNHB", "PNHAI", "PNHA", "PNHNH", "PNHO", "PNHT", "PHISP",
    "MEDAGE"
]

md_tracts = md[md_cols_keep].copy()
md_tracts.head()


,GEOID,SQ_MILES,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,MEDAGE
0,242402924029950100,66.23845,3838,1796,3143,299,5,37,0,2,47,305,81.9,7.8,0.1,1.0,0.0,0.1,1.2,7.9,42.8
1,242402924029950200,108.66000,4844,2640,3767,737,4,14,0,4,58,260,77.8,15.2,0.1,0.3,0.0,0.1,1.2,5.4,46.2
2,242402924029950300,8.33238,5775,2574,4226,1085,5,95,1,9,107,247,73.2,18.8,0.1,1.6,0.0,0.2,1.9,4.3,36.9
3,242402924029950500,28.43434,2623,1770,2307,236,2,4,0,0,35,39,88.0,9.0,0.1,0.2,0.0,0.0,1.3,1.5,52.3
4,242402924029950400,67.97242,3117,1769,2340,646,7,7,2,3,56,56,75.1,20.7,0.2,0.2,0.1,0.1,1.8,1.8,50.9


In [79]:
tracts = tracts.merge(md_tracts, on="GEOID", how="left")

print("Tracts with population data:",
      tracts["POP100"].notna().sum(), "of", len(tracts))

tracts.head()


Tracts with population data: 0 of 1475


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,total_pop_x,SQ_MILES_x,POP100_x,HU100_x,NHW_x,NHB_x,NHAI_x,NHA_x,NHNH_x,NHO_x,NHT_x,HISP_x,PNHW_x,PNHB_x,PNHAI_x,PNHA_x,PNHNH_x,PNHO_x,PNHT_x,PHISP_x,MEDAGE_x,OBJECTID_1,FID_TL_201,ALAND10,AWATER10,CNTY2010,CT10,CT2010,GEODESC,ACRES,SQ_MILES_y,POP100_y,HU100_y,NHW_y,NHB_y,NHAI_y,NHA_y,NHNH_y,NHO_y,NHT_y,HISP_y,PNHW_y,PNHB_y,PNHAI_y,PNHA_y,PNHNH_y,PNHO_y,PNHT_y,PHISP_y,POP65_,PCTPOP65_,MEDAGE_y,VACNS,PVACNS,PHOWN,PWOMORT,PRENT,PLT18SP,CNTY20101,CT101,AREA,ACRES_1,SQMILES,PSQM,REPORT_2_P,REPORT_9_P,Shape_Length,Shape_Area,population,total_pop_y,total_pop,SQ_MILES,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,MEDAGE
0,24,005,403100,24005403100,1400000US24005403100,4031,Census Tract 4031,G5020,S,1913598,0,+39.3530950,-076.7333148,"POLYGON ((-76.74491 39.35986, -76.74431 39.359...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,492.0,496.0,1913333.0,0.0,24005.0,403100.0,2.400540e+10,4031.00,472.77936,0.73872,2462.0,952.0,720.0,1591.0,4.0,22.0,0.0,7.0,44.0,74.0,29.2,64.6,0.2,0.9,0.0,0.3,1.8,3.0,730.0,29.7,51.6,36.0,3.8,83.4,23.2,16.6,40.0,24005.0,403100.0,0.0,472.77936,0.73872,3332.81,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,9614.294520,3.204235e+06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,005,403201,24005403201,1400000US24005403201,4032.01,Census Tract 4032.01,G5020,S,1764534,0,+39.3549038,-076.7215235,"POLYGON ((-76.73369 39.36204, -76.7329 39.3620...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,549.0,553.0,1764535.0,0.0,24005.0,403201.0,2.400540e+10,4032.01,436.01019,0.68127,3076.0,1191.0,281.0,2563.0,24.0,33.0,2.0,3.0,65.0,105.0,9.1,83.3,0.8,1.1,0.1,0.1,2.1,3.4,481.0,15.6,44.2,47.0,3.9,92.5,18.3,7.5,48.3,24005.0,403201.0,0.0,436.01019,0.68127,4515.13,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,8352.671252,2.955185e+06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,033,807304,24033807304,1400000US24033807304,8073.04,Census Tract 8073.04,G5020,S,1776714,0,+39.0249780,-076.9594342,"POLYGON ((-76.97171 39.01858, -76.97164 39.018...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,769.0,773.0,1779313.0,0.0,24033.0,807304.0,2.403381e+10,8073.04,439.63469,0.68693,1928.0,625.0,462.0,522.0,9.0,169.0,0.0,2.0,62.0,702.0,24.0,27.1,0.5,8.8,0.0,0.1,3.2,36.4,291.0,15.1,39.2,37.0,5.9,91.3,23.1,8.7,24.7,24033.0,807304.0,0.0,439.63469,0.68693,2806.70,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,7884.644364,2.952312e+06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24,033,807305,24033807305,1400000US24033807305,8073.05,Census Tract 8073.05,G5020,S,3030479,4428,+39.0121779,-076.9635510,"POLYGON ((-76.97728 39.01095, -76.97714 39.011...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453.0,457.0,3078040.0,4428.0,24033.0,807305.0,2.403381e+10,8073.05,761.61967,1.19003,3676.0,1116.0,478.0,1230.0,8.0,400.0,3.0,10.0,70.0,1477.0,13.0,33.5,0.2,10.9,0.1,0.3,1.9,40.2,356.0,9.7,33.1,52.0,4.7,64.9,16.0,35.1,31.1,24033.0,807305.0,0.0,761.61967,1.19003,3089.00,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,9375.156855,5.112749e+06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,033,807405,24033807405,1400000US24033807405,8074.05,Census Tract 8074.05,G5020,S,7093435,27200,+39.0383526,-076.9283090,"POLYGON ((-76.94986 39.02208, -76.94944 39.024...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,768.0,772.0,7092548.0,27

In [81]:
STATE_FIPS = "24"  # Maryland

# county: 3-digit FIPS
md["COUNTYFP"] = md["CNTY2010"].astype(str).str.zfill(3)

# tract: strip decimals (e.g. 803.01 → '80301’) and pad to 6 digits
tract_raw = md["CT2010"].astype(str).str.strip()
tract_clean = tract_raw.str.replace(".", "", regex=False)
md["TRACTCE"] = tract_clean.str.zfill(6)

# final 11-digit GEOID
md["GEOID"] = STATE_FIPS + md["COUNTYFP"] + md["TRACTCE"]

md[["CNTY2010", "CT2010", "COUNTYFP", "TRACTCE", "GEOID"]].head()


,CNTY2010,CT2010,COUNTYFP,TRACTCE,GEOID
0,24029,24029950100,24029,24029950100,242402924029950100
1,24029,24029950200,24029,24029950200,242402924029950200
2,24029,24029950300,24029,24029950300,242402924029950300
3,24029,24029950500,24029,24029950500,242402924029950500
4,24029,24029950400,24029,24029950400,242402924029950400


In [82]:
md_cols_keep = [
    "GEOID",
    "SQ_MILES",
    "POP100", "HU100",
    "NHW", "NHB", "NHAI", "NHA", "NHNH", "NHO", "NHT", "HISP",
    "PNHW", "PNHB", "PNHAI", "PNHA", "PNHNH", "PNHO", "PNHT", "PHISP",
    "MEDAGE"
]

md_tracts = md[md_cols_keep].copy()
md_tracts.head()


,GEOID,SQ_MILES,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,MEDAGE
0,242402924029950100,66.23845,3838,1796,3143,299,5,37,0,2,47,305,81.9,7.8,0.1,1.0,0.0,0.1,1.2,7.9,42.8
1,242402924029950200,108.66000,4844,2640,3767,737,4,14,0,4,58,260,77.8,15.2,0.1,0.3,0.0,0.1,1.2,5.4,46.2
2,242402924029950300,8.33238,5775,2574,4226,1085,5,95,1,9,107,247,73.2,18.8,0.1,1.6,0.0,0.2,1.9,4.3,36.9
3,242402924029950500,28.43434,2623,1770,2307,236,2,4,0,0,35,39,88.0,9.0,0.1,0.2,0.0,0.0,1.3,1.5,52.3
4,242402924029950400,67.97242,3117,1769,2340,646,7,7,2,3,56,56,75.1,20.7,0.2,0.2,0.1,0.1,1.8,1.8,50.9


In [88]:
tracts = tracts.merge(md_tracts, on="GEOID", how="left")

# Inspect columns that look like population
print("All columns:", tracts.columns.tolist())
print("Population-ish columns:", [c for c in tracts.columns if "POP" in c.upper()])


MergeError: Passing 'suffixes' which cause duplicate columns {'PNHW_x', 'PNHT_x', 'PNHB_x', 'PNHA_x', 'NHAI_x', 'NHT_x', 'POP100_x', 'NHB_x', 'PHISP_x', 'PNHO_x', 'HU100_x', 'NHNH_x', 'PNHAI_x', 'NHO_x', 'MEDAGE_x', 'HISP_x', 'NHW_x', 'NHA_x', 'SQ_MILES_x', 'PNHNH_x'} is not allowed.

Standardize GEOIDs in both datasets

In [56]:
tracts['GEOID'] = tracts['GEOID'].astype(str).str.zfill(11)
md['GEOID'] = md['CNTY20101'].astype(str).str.zfill(5) + md['CT101'].astype(str).str.zfill(6)
race = race.rename(columns={"Label (Grouping)": "label"})
race['GEOID'] = race['label'].str.extract(r'(\d{11})')  # optional if race is used later


In [57]:
tracts = tracts.merge(md, on="GEOID", how="left")


In [61]:
# Columns we want to use, in priority order
pop_cols = ["total_pop", "POP100", "POP100_x", "POP100_y"]

# Keep only the ones that actually exist in this DataFrame
pop_cols = [c for c in pop_cols if c in tracts.columns]

# For each row, take the first non-NaN across these columns
temp = tracts[pop_cols].bfill(axis=1)
tracts["population"] = temp.iloc[:, 0]

print("Tracts with population:", tracts["population"].notna().sum(), "of", len(tracts))
tracts[["GEOID", "NAME", "population"]].head()



Tracts with population: 1273 of 1475


,GEOID,NAME,population
0,24005403100,4031,2462.0
1,24005403201,4032.01,3076.0
2,24033807304,8073.04,1928.0
3,24033807305,8073.05,3676.0
4,24033807405,8074.05,5858.0


In [ ]:
# Create a clean population column using the MD dataset (POP100_y)
tracts["population"] = tracts["POP100_y"].fillna(0).astype(int)

print("Tracts with population:", tracts["population"].gt(0).sum(), "of", len(tracts))
print("Total Alt-Baltimore population:", tracts["population"].sum())
tracts.head()


Tracts with population: 1270 of 1475
Total Alt-Baltimore population: 5069540


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,total_pop_x,SQ_MILES_x,POP100_x,HU100_x,NHW_x,NHB_x,NHAI_x,NHA_x,NHNH_x,NHO_x,NHT_x,HISP_x,PNHW_x,PNHB_x,PNHAI_x,PNHA_x,PNHNH_x,PNHO_x,PNHT_x,PHISP_x,MEDAGE_x,OBJECTID_1,FID_TL_201,ALAND10,AWATER10,CNTY2010,CT10,CT2010,GEODESC,ACRES,SQ_MILES_y,POP100_y,HU100_y,NHW_y,NHB_y,NHAI_y,NHA_y,NHNH_y,NHO_y,NHT_y,HISP_y,PNHW_y,PNHB_y,PNHAI_y,PNHA_y,PNHNH_y,PNHO_y,PNHT_y,PHISP_y,POP65_,PCTPOP65_,MEDAGE_y,VACNS,PVACNS,PHOWN,PWOMORT,PRENT,PLT18SP,CNTY20101,CT101,AREA,ACRES_1,SQMILES,PSQM,REPORT_2_P,REPORT_9_P,Shape_Length,Shape_Area,population,total_pop_y,total_pop,SQ_MILES,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,MEDAGE
0,24,005,403100,24005403100,1400000US24005403100,4031,Census Tract 4031,G5020,S,1913598,0,+39.3530950,-076.7333148,"POLYGON ((-76.74491 39.35986, -76.74431 39.359...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,492.0,496.0,1913333.0,0.0,24005.0,403100.0,2.400540e+10,4031.00,472.77936,0.73872,2462.0,952.0,720.0,1591.0,4.0,22.0,0.0,7.0,44.0,74.0,29.2,64.6,0.2,0.9,0.0,0.3,1.8,3.0,730.0,29.7,51.6,36.0,3.8,83.4,23.2,16.6,40.0,24005.0,403100.0,0.0,472.77936,0.73872,3332.81,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,9614.294520,3.204235e+06,2462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,005,403201,24005403201,1400000US24005403201,4032.01,Census Tract 4032.01,G5020,S,1764534,0,+39.3549038,-076.7215235,"POLYGON ((-76.73369 39.36204, -76.7329 39.3620...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,549.0,553.0,1764535.0,0.0,24005.0,403201.0,2.400540e+10,4032.01,436.01019,0.68127,3076.0,1191.0,281.0,2563.0,24.0,33.0,2.0,3.0,65.0,105.0,9.1,83.3,0.8,1.1,0.1,0.1,2.1,3.4,481.0,15.6,44.2,47.0,3.9,92.5,18.3,7.5,48.3,24005.0,403201.0,0.0,436.01019,0.68127,4515.13,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,8352.671252,2.955185e+06,3076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,033,807304,24033807304,1400000US24033807304,8073.04,Census Tract 8073.04,G5020,S,1776714,0,+39.0249780,-076.9594342,"POLYGON ((-76.97171 39.01858, -76.97164 39.018...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,769.0,773.0,1779313.0,0.0,24033.0,807304.0,2.403381e+10,8073.04,439.63469,0.68693,1928.0,625.0,462.0,522.0,9.0,169.0,0.0,2.0,62.0,702.0,24.0,27.1,0.5,8.8,0.0,0.1,3.2,36.4,291.0,15.1,39.2,37.0,5.9,91.3,23.1,8.7,24.7,24033.0,807304.0,0.0,439.63469,0.68693,2806.70,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,7884.644364,2.952312e+06,1928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24,033,807305,24033807305,1400000US24033807305,8073.05,Census Tract 8073.05,G5020,S,3030479,4428,+39.0121779,-076.9635510,"POLYGON ((-76.97728 39.01095, -76.97714 39.011...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453.0,457.0,3078040.0,4428.0,24033.0,807305.0,2.403381e+10,8073.05,761.61967,1.19003,3676.0,1116.0,478.0,1230.0,8.0,400.0,3.0,10.0,70.0,1477.0,13.0,33.5,0.2,10.9,0.1,0.3,1.9,40.2,356.0,9.7,33.1,52.0,4.7,64.9,16.0,35.1,31.1,24033.0,807305.0,0.0,761.61967,1.19003,3089.00,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,9375.156855,5.112749e+06,3676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,033,807405,24033807405,1400000US24033807405,8074.05,Census Tract 8074.05,G5020,S,7093435,27200,+39.0383526,-076.9283090,"POLYGON ((-76.94986 39.02208, -76.94944 39.024...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,768.0,772.0,

In [68]:
print("Tracts GEOIDs:", tracts["GEOID"].head().tolist())
print("MD GEOIDs:", md["GEOID"].head().tolist())


Tracts GEOIDs: ['24005403100', '24005403201', '24033807304', '24033807305', '24033807405']
MD GEOIDs: ['24029950100', '24029950200', '24029950300', '24029950500', '24029950400']
